In [1]:
import keras
import pandas as pd
from keras import layers
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
import numpy as np

2025-01-05 03:53:23.903352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736049203.912793  558230 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736049203.915642  558230 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 03:53:23.925941: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Dense, InputLayer, Bidirectional,LSTM, Dropout, SimpleRNN, GRU
# from keras import ops
import numpy as np

In [3]:
DC_POWER_INDEX = 0
LEARNING_RATE = 0.001
RANDOM_STATE = 44
START_TIME = '2020-06-07 00:30:00'
END_TIME = '2020-06-07 04:00:00'
NUMBER_OF_FEATURES = 8
BATCH_SIZE = 1024
# Training window
MULTIV_TRAINING_WINDOW = 8

def fix_randomness():
    tf.random.set_seed(RANDOM_STATE)

    from numpy.random import seed
    seed(RANDOM_STATE)
    keras.utils.set_random_seed(RANDOM_STATE)

fix_randomness()

MULTIVARIATE

In [4]:
EPOCHS = 500
MULTIV_TRAINING_WINDOW = 40
lead_time = 20

In [5]:
# Creating the model input for multivariate forecasting
def df_to_model_input2(df_np, col_index, window_size):
    df_np = df_np.to_numpy()
    x = []
    y = []

    for i in range(len(df_np) - window_size - lead_time):
        row = [r for r in df_np[i:i+window_size]]
        x.append(row)

        label = df_np[i+lead_time+window_size][col_index]
        y.append(label)

    return np.array(x), np.array(y)

In [6]:
multiv_teacher_path = 'models/girasol/multiv_teacher_model.keras'
multiv_student_path = 'models/girasol/multiv_student_model.keras'
distilled_student_path = 'models/girasol/distilled_multiv_student_model.keras'

In [7]:
train = pd.read_csv('/.../girasol_met/train.csv')
test = pd.read_csv('/.../girasol_met/val.csv')

In [8]:
NUMBER_OF_FEATURES = len(train.columns)

In [9]:
scaler = MinMaxScaler()
train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)
test = pd.DataFrame(scaler.transform(test), columns=test.columns)

In [10]:
def get_train_test_val(x, y):
    return x[:train_len], y[:train_len], x[train_len:], y[train_len:]

x, y = df_to_model_input2(train, 7, MULTIV_TRAINING_WINDOW)

ds_len = len(y)
train_len = int(0.8*ds_len)

x_train, y_train, x_val, y_val = get_train_test_val(x, y)
x_test, y_test = df_to_model_input2(test, 7, MULTIV_TRAINING_WINDOW)

In [11]:
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def calculate_mape(scaler, y_pred_scaled, target_column_index=-1):

    # Create dummy arrays to match the scaler's expected input shape
    dummy_data_pred = np.zeros((y_pred_scaled.shape[0], NUMBER_OF_FEATURES))
    dummy_data_test = np.zeros((y_test.shape[0], NUMBER_OF_FEATURES))

    # Insert the predicted values and y_test into the correct column
    dummy_data_pred[:, target_column_index] = y_pred_scaled[:, 0]
    dummy_data_test[:, target_column_index] = y_test[:] # it kept that way to be able to modify if needed

    # Perform inverse transform to get the unscaled predictions and y_test
    unscaled_predictions = scaler.inverse_transform(dummy_data_pred)[:, target_column_index]
    y_test_unscaled = scaler.inverse_transform(dummy_data_test)[:, target_column_index]



    mape = mean_absolute_percentage_error(y_test_unscaled, unscaled_predictions)
    return mape

def calculate_metrics(y_true, y_pred):
    """
    Calculate NMSE, NRMSE, and NMAE normalized by the average of true values.
    
    Parameters:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.
        
    Returns:
        dict: A dictionary containing NMSE, NRMSE, and NMAE.
    """
    # Convert inputs to numpy arrays for consistency
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Mean Squared Error (MSE) and Mean Absolute Error (MAE) using sklearn
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    mape = calculate_mape(scaler=scaler, y_pred_scaled=y_pred)*100
    # Variance and mean of true values
    avg_true = np.mean(y_true)
    
    # Calculate metrics
    nrmse = np.sqrt(mse) / avg_true
    nmae = mae / avg_true

    print(f"NRMSE (Normalized Root Mean Squared Error): {nrmse:.4f}")
    print(f"NMAE (Normalized Mean Absolute Error): {nmae:.4f}")
    print(f"MAPE (Mean Absolute Percentage Error): {mape:.4f}") 
    
    # Return metrics as a dictionary
    return nrmse, nmae, mape

In [12]:
columns = ['Hidden Units', 'Dense Units', 'RMSE']
results_df = pd.DataFrame(columns=columns)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import os

# Check if the 'results' directory exists in the current working directory
if not os.path.exists('results'):
    # If the directory does not exist, you can create it
    os.makedirs('results')

script_name = 'results/multiv_girasol_analysis'
def run_student_multiv(hidden_unit, dense_param):

    global results_df  # To modify the global DataFrame

    # Create the model
    model = Sequential()
    model.add(InputLayer((MULTIV_TRAINING_WINDOW, NUMBER_OF_FEATURES)))
    model.add(GRU(hidden_unit))
    model.add(Dropout(0.5))
    model.add(Dense(dense_param, 'relu'))
    model.add(Dense(1, 'linear'))

    # Model checkpoint
    model_file = multiv_student_path

    cp = ModelCheckpoint(model_file, save_best_only=True)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restore the weights of the best epoch after stopping
    )

    # Compile the model
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=[MeanSquaredError()]
    )
   
    # Train the model
    model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[cp, early_stopping]
    )

    # Make predictions on the scaled test set
    y_pred = model.predict(x_test)

    nrmse, nmae, mape = calculate_metrics(y_test, y_pred)

    # Add a new row to the DataFrame with all metrics
    new_row = pd.DataFrame({
        'Hidden Units': [hidden_unit], 
        'Dense Units': [dense_param], 
        'NRMSE': [nrmse], 
        'NMAE': [nmae], 
        'MAPE': [mape]
    })

    # Concatenate the new row to the results DataFrame
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Save the DataFrame to a CSV file after each run
    results_df.to_csv(f'{script_name}.csv', index=False)

    calculate_mape(scaler=scaler, y_pred_scaled=y_pred)

    # Return a summary string with all the metrics
    return f"Hidden Units: {hidden_unit}, Dense Units: {dense_param}, NRMSE: {nrmse:.5f}, NMAE: {nmae:.5f}, MAPE: {mape:.5f}"


hidden_units = [16, 32, 64, 128, 512]
dense_params = [8]

rmse_list = []
for i in hidden_units:
    for j in dense_params:
        RANDOM_STATE = 44
        tf.random.set_seed(RANDOM_STATE)
        from numpy.random import seed
        seed(RANDOM_STATE)
        keras.utils.set_random_seed(RANDOM_STATE)

        s = run_student_multiv(i,j)
        rmse_list.append(s)

rmse_list

I0000 00:00:1736049206.758568  558230 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/500


I0000 00:00:1736049208.323390  558320 cuda_dnn.cc:529] Loaded cuDNN version 90300


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.1493 - mean_squared_error: 0.1492 - val_loss: 0.0366 - val_mean_squared_error: 0.0374
Epoch 2/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0571 - mean_squared_error: 0.0571 - val_loss: 0.0267 - val_mean_squared_error: 0.0287
Epoch 3/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0406 - mean_squared_error: 0.0406 - val_loss: 0.0188 - val_mean_squared_error: 0.0202
Epoch 4/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0326 - mean_squared_error: 0.0326 - val_loss: 0.0142 - val_mean_squared_error: 0.0147
Epoch 5/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0265 - mean_squared_error: 0.0264 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 6/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0218 - mean_squared_error: 0.0218 - val_loss: 0.0086 - val_mean_squared_error: 0.0084
Epoch 7/500
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0185 - mean_squared_error: 0.0185 - val_loss: 0.0081 - val_mea

['Hidden Units: 16, Dense Units: 8, NRMSE: 0.15101, NMAE: 0.09783, MAPE: 11.76204',
 'Hidden Units: 16, Dense Units: 16, NRMSE: 0.30787, NMAE: 0.27646, MAPE: 26.12429',
 'Hidden Units: 32, Dense Units: 8, NRMSE: 0.15879, NMAE: 0.11181, MAPE: 12.60192',
 'Hidden Units: 32, Dense Units: 16, NRMSE: 0.15907, NMAE: 0.11306, MAPE: 12.48550',
 'Hidden Units: 64, Dense Units: 8, NRMSE: 0.13630, NMAE: 0.07839, MAPE: 10.51338',
 'Hidden Units: 64, Dense Units: 16, NRMSE: 0.13540, NMAE: 0.07784, MAPE: 10.32301',
 'Hidden Units: 128, Dense Units: 8, NRMSE: 0.13851, NMAE: 0.08057, MAPE: 10.33352',
 'Hidden Units: 128, Dense Units: 16, NRMSE: 0.15621, NMAE: 0.10840, MAPE: 12.58312',
 'Hidden Units: 512, Dense Units: 8, NRMSE: 0.13556, NMAE: 0.07560, MAPE: 10.02053',
 'Hidden Units: 512, Dense Units: 16, NRMSE: 0.13595, NMAE: 0.07611, MAPE: 10.13766']

In [14]:
rmse_list

['Hidden Units: 16, Dense Units: 8, NRMSE: 0.15101, NMAE: 0.09783, MAPE: 11.76204',
 'Hidden Units: 16, Dense Units: 16, NRMSE: 0.30787, NMAE: 0.27646, MAPE: 26.12429',
 'Hidden Units: 32, Dense Units: 8, NRMSE: 0.15879, NMAE: 0.11181, MAPE: 12.60192',
 'Hidden Units: 32, Dense Units: 16, NRMSE: 0.15907, NMAE: 0.11306, MAPE: 12.48550',
 'Hidden Units: 64, Dense Units: 8, NRMSE: 0.13630, NMAE: 0.07839, MAPE: 10.51338',
 'Hidden Units: 64, Dense Units: 16, NRMSE: 0.13540, NMAE: 0.07784, MAPE: 10.32301',
 'Hidden Units: 128, Dense Units: 8, NRMSE: 0.13851, NMAE: 0.08057, MAPE: 10.33352',
 'Hidden Units: 128, Dense Units: 16, NRMSE: 0.15621, NMAE: 0.10840, MAPE: 12.58312',
 'Hidden Units: 512, Dense Units: 8, NRMSE: 0.13556, NMAE: 0.07560, MAPE: 10.02053',
 'Hidden Units: 512, Dense Units: 16, NRMSE: 0.13595, NMAE: 0.07611, MAPE: 10.13766']